In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

In [1]:
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import scipy.stats

from jax import pure_callback, random
import jax.numpy as jnp
import optax

import numpyro
from numpyro import handlers
from numpyro.contrib.funsor import config_enumerate, infer_discrete
import numpyro.distributions as dist
from numpyro.distributions import constraints
from numpyro.infer import SVI, TraceEnum_ELBO, init_to_value
from numpyro.infer.autoguide import AutoDelta

/Users/jaminkim/anaconda3/envs/pymc_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre-Processing
1. Import players who have over 400 plate appearances in 2024.
2. Import play-by-play data from Statcast in 2024.
3. Filtered specific columns to keep in play-by-play data.
4. Filtered bunts from play-by-play data.
5. Remove NA's from needed columns

In [5]:
#Get players who have 400 plate appearances or more
from baseball_processing.data import *
fangraphs_batting_min_400_2024 = get_table("fangraphs_batting_min_400_2024")

In [3]:
swing_2024 = get_swing_data([2024])

2024: 322559 rows


In [10]:
from baseball_processing.preprocessing import *
columns_to_keep = ["pitch_type", 
                     "game_date", 
                     "release_speed", 
                     "player_name", 
                     "batter", 
                     "pitcher", 
                     "events",
                     "description",
                     "des", 
                     "zone",
                     "game_type",
                     "stand",
                     "p_throws",
                     "bb_type",
                     "balls",
                     "strikes",
                     "game_year",
                     "plate_x",
                     "plate_z",
                     "launch_speed",
                     "launch_angle",
                     "effective_speed",
                     "release_spin_rate",
                     "release_extension", 
                     "estimated_ba_using_speedangle",
                     "estimated_woba_using_speedangle",
                     "woba_value",
                     "arm_angle",
                     "bat_speed",
                     "swing_length"]
columns_na = ["bat_speed", "swing_length"]
columns_not_na = remove_nan(swing_2024, columns_na)
swing_2024 = remove_columns(swing_2024, columns_to_keep)


No rows dropped.


In [8]:
mlbam_ids_2024 = fangraphs_batting_min_400_2024["mlbam_id"]
swing_2024 = swing_2024[swing_2024["batter"].isin(mlbam_ids_2024)]

In [9]:
swing_2024 = swing_2024[(swing_2024["bat_speed"] > 20) & (swing_2024["swing_length"] > 3)]

## Modeling